<a href="https://colab.research.google.com/github/Mufid99/Population_Coding_Thesis/blob/master/Source_code_dissertation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import time

import csv
import operator
import pandas as pd
import numpy as np
import itertools
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.mixture import GaussianMixture
from sklearn.neural_network import MLPRegressor
import numbers
import functools
import operator

'''
Insert any of these datasets into the datasets array:

'1027_ESL', '1028_SWD', '1029_LEV', '1030_ERA', '1089_USCrime', 
'1096_FacultySalaries', '192_vineyard', '195_auto_price', '207_autoPrice', 
'210_cloud', '228_elusage', '230_machine_cpu', '485_analcatdata_vehicle', 
'519_vinnie', '522_pm10', '523_analcatdata_neavote', 
'527_analcatdata_election2000', '542_pollution', '547_no2', 
'556_analcatdata_apnea2', '557_analcatdata_apnea1', '561_cpu', 
'579_fri_c0_250_5', '581_fri_c3_500_25', '582_fri_c1_500_25', 
'583_fri_c1_1000_50', '584_fri_c4_500_25', '586_fri_c3_1000_25', 
'588_fri_c4_1000_100', '589_fri_c2_1000_25', '590_fri_c0_1000_50'
, '591_fri_c1_100_10', '592_fri_c4_1000_25', '593_fri_c1_1000_10', 
'594_fri_c2_100_5', '595_fri_c0_1000_10', '596_fri_c2_250_5', 
'597_fri_c2_500_5', '598_fri_c0_1000_25', '599_fri_c2_1000_5', 
'601_fri_c1_250_5', '602_fri_c3_250_10', '603_fri_c0_250_50', 
'604_fri_c4_500_10', '605_fri_c2_250_25', '606_fri_c2_1000_10', 
'607_fri_c4_1000_50', '608_fri_c3_1000_10', '609_fri_c0_1000_5', 
'611_fri_c3_100_5', '612_fri_c1_1000_5', '613_fri_c3_250_5', 
'615_fri_c4_250_10', '616_fri_c4_500_50', '617_fri_c3_500_5', 
'618_fri_c3_1000_50', '620_fri_c1_1000_25', '621_fri_c0_100_10', 
'622_fri_c2_1000_50', '623_fri_c4_1000_10', '624_fri_c0_100_5', 
'626_fri_c2_500_50', '627_fri_c2_500_10', '628_fri_c3_1000_5', 
'631_fri_c1_500_5', '633_fri_c0_500_25', '634_fri_c2_100_10', 
'635_fri_c0_250_10', '637_fri_c1_500_50', '641_fri_c1_500_10', 
'643_fri_c2_500_25', '644_fri_c4_250_25', '645_fri_c3_500_50', 
'646_fri_c3_500_10', '647_fri_c1_250_10', '648_fri_c1_250_50', 
'649_fri_c0_500_5', '650_fri_c0_500_50', '651_fri_c0_100_25', 
'653_fri_c0_250_25', '654_fri_c0_500_10', '656_fri_c1_100_5', 
'657_fri_c2_250_10', '658_fri_c3_250_25', '659_sleuth_ex1714', 
'663_rabe_266', '665_sleuth_case2002', '666_rmftsa_ladata', 
'678_visualizing_environmental', '687_sleuth_ex1605', '690_visualizing_galaxy', 
'695_chatfield_4', '706_sleuth_case1202', '712_chscase_geyser1'
'''

# datasets to benchmark the algorithm against
# any combination of the above datasets can be inserted into this array
datasets = ['601_fri_c1_250_5', '656_fri_c1_100_5']

# turn input coding and output coding on and off
CODE_INPUTS = True
CODE_OUTPUTS = True

# select of number of Gaussian centres for inputs or outputs
# if either of the two constants above are false, then the corresponding number 
# of centres are not used in the code below
NUM_GAUSSIAN_CENTRES_X = 7
NUM_GAUSSIAN_CENTRES_Y = 10


# Transforms dataset string into url
def datasets_to_urls(datasets):
  left_part = "https://github.com/EpistasisLab/penn-ml-benchmarks/blob/master/datasets/regression/"
  right_part = ".tsv.gz?raw=true"
  urls = [left_part + dataset + "/" + dataset + right_part for dataset in datasets]
  return urls

# calculates Gaussian centres and standard deviations of dataset using a
# a gaussian mixture model and returns both
def get_gmm_gcs_stds(data, num_gcs):
  gaussian_centres = []
  stds = []
  # loops over each feature
  for i in range(len(data[0])):
    num_comp = num_gcs
    # checks the number of unique datapoints in a feature
    unique_comp_len = len(np.unique(data[:,i]))
    # if the number of unique datapoints are less than the number of components
    # then the number of Gaussian centres is set equal to the number of datapoints
    if unique_comp_len < num_comp:
      num_comp = unique_comp_len
    gm = GaussianMixture(n_components=num_comp)
    gm.fit(data[:,i].reshape(-1,1))
    # adjusting the dimensions of the arrays
    means = functools.reduce(operator.iconcat,gm.means_,[])
    variances = gm.covariances_.flatten()
    # remove centres close in value
    means, variances = remove_centres(means, variances, np.ptp(data[:,i]))
    gaussian_centres.append(np.array(means))
    stds.append(np.sqrt(variances))
  return gaussian_centres, stds

# removes centres that are close in value to each other
def remove_centres(means, variances, values_range):
  index = 0
  while index < len(means):
    index2 = index + 1
    while index2 < len(means):
      # if two means are within a tenth of the range
      if abs(means[index] - means[index2]) <= (0.1 * values_range):
        means.pop(index2)
        variances = np.delete(variances, index2)
      index2 += 1
    index += 1
  return means, variances

# calculates gaussian centres using the intervals of the range of a dataset also 
# calculates the standard deviation of the dataset and returns both
def get_interval_gcs_std(data, n_cent):
  data_range = np.ptp(data)
  # the difference between 2 consecutive intervals
  increm = data_range/(n_cent-1)
  # the first centre is the minumum value in the range
  cur_cen = np.min(data)
  gaussian_centres = []
  for i in range(n_cent):
    gaussian_centres.append(cur_cen)
    cur_cen += increm
  std = np.std(data)
  return [np.array(gaussian_centres)], [std]

# given a set of gaussian centres and standard deviations,
# codes a value into multiple new values representing new features
def code(mu, s, sigma):
  # if mu is not a number a row with as many 0s as the number of centres is returned
  if not isinstance(mu, numbers.Number):
      return np.zeros((1, len(s)))
  # checks if there are multiple standard deviations
  if isinstance(sigma, (list, np.ndarray)):
    for i in sigma:
      # replace std with 1e-6 if it is smaller
      i = max(1e-6, i)
  else:
    sigma = max(1e-6, sigma)
  p1 = np.divide(0.5, (np.square(sigma))) 
  p2 = np.subtract(mu, s)
  p3 = np.square(p2)
  p4 = np.multiply(-p1, p3)
  z = np.exp(p4)
  return z

# Applies coding to dataset passed using the Gaussian centres and stds
def code_data(data, gaussian_centres, stds):
  # new coded data of the neural network (new X or y)
  coded_data = []
  # each row in the dataset
  for m in range(len(data)):
      coded_data.append([])
      # each column
      for i in range(len(data[m])):
          # makes sure there is more than 1 centre to apply coding
          if gaussian_centres[i].size > 1:
              coded = code(data[m][i], gaussian_centres[i], stds[i])
              for j in coded:
                  coded_data[m].append(j)
          else:
              coded_data[m].append(data[m][i])
  return coded_data

# decodes data to original value given the gaussian 
# centres used to code the data
def decode(data, gaussian_centres):
  new_data = []
  for i in range(len(data)):
    total = np.sum(data[i])
    prod = np.sum(np.multiply(data[i], gaussian_centres[0]))
    new_data.append([prod/total])
  return np.array(new_data)

urls = datasets_to_urls(datasets) 

'''
Code beyond this point that does not involve population coding (calling the above
functions) or writing to a csv is borrowed from the regression-benchmark repository.

Author: Orzechowski, Patryk 
Date: 2018
https://github.com/EpistasisLab/regression-benchmark/blob/master/scikit/validation-MLPRegressor.py
'''

for url in urls:
  input_data = pd.read_csv(url, compression='gzip', sep='\t')

  TARGET_NAME = 'target'
  INPUT_SEPARATOR = '\t'
  n_jobs = 1
    
  hyper_params = [
      {
          'activation' : ('logistic', 'tanh', 'relu',),
          'solver' : ('lbfgs','adam','sgd',),
          'learning_rate' : ('constant', 'invscaling', 'adaptive',),
      },
  ]

  sc_y = StandardScaler()
  X = StandardScaler().fit_transform(input_data.drop(TARGET_NAME, axis=1).values.astype(float))
  y = sc_y.fit_transform(input_data[TARGET_NAME].values.reshape(-1,1))

  data_titles = ["Trial Number", "Best Params", "Train Score mse", 
                 "Train Score mae", "Test Score mse", "Test Score mae", "Runtime"]
  with open(url.split('/')[-1][:-16]+ ".csv", 'w', newline='') as file:
      wr = csv.writer(file)
      wr.writerow(data_titles)

  for i in range(1, 11):

    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                        train_size=0.75,
                                                        test_size=0.25,
                                                        random_state=None)
    
    if CODE_INPUTS:
      gaussian_centres_X, stds_X = get_gmm_gcs_stds(X_train, NUM_GAUSSIAN_CENTRES_X)
      X_train = np.array(code_data(X_train, gaussian_centres_X, stds_X))
      X_test = np.array(code_data(X_test, gaussian_centres_X, stds_X))
    
    if CODE_OUTPUTS:
      gaussian_centres_y, stds_y = get_interval_gcs_std(y_train, NUM_GAUSSIAN_CENTRES_Y)
      y_train_coded = np.array(code_data(y_train, gaussian_centres_y, stds_y))  

    est=MLPRegressor()
    grid_clf = GridSearchCV(est,cv=5,param_grid=hyper_params,
                      verbose=0,n_jobs=n_jobs,scoring='r2')

    t0 = time.time()
    if CODE_OUTPUTS:
      #fit model
      grid_clf.fit(X_train, y_train_coded)
      #get fit time
      runtime = time.time()-t0
      train_score_mse = mean_squared_error(sc_y.inverse_transform(y_train),
      sc_y.inverse_transform
      (decode(grid_clf.predict(X_train), gaussian_centres_y)))
      train_score_mae = mean_absolute_error(sc_y.inverse_transform(y_train),
      sc_y.inverse_transform
      (decode(grid_clf.predict(X_train), gaussian_centres_y)))
      test_score_mse = mean_squared_error(sc_y.inverse_transform(y_test),
      sc_y.inverse_transform
      (decode(grid_clf.predict(X_test), gaussian_centres_y)))
      test_score_mae = mean_absolute_error(sc_y.inverse_transform(y_test),
      sc_y.inverse_transform
      (decode(grid_clf.predict(X_test), gaussian_centres_y)))
    else:
      #fit model
      grid_clf.fit(X_train,y_train.ravel())
      #get fit time
      runtime = time.time()-t0
      train_score_mse = mean_squared_error(sc_y.inverse_transform(y_train),
      sc_y.inverse_transform(grid_clf.predict(X_train)))
      train_score_mae = mean_absolute_error(sc_y.inverse_transform(y_train),
      sc_y.inverse_transform(grid_clf.predict(X_train)))
      test_score_mse = mean_squared_error(sc_y.inverse_transform(y_test),
      sc_y.inverse_transform(grid_clf.predict(X_test)))
      test_score_mae = mean_absolute_error(sc_y.inverse_transform(y_test),
      sc_y.inverse_transform(grid_clf.predict(X_test)))

    sorted_grid_params = sorted(grid_clf.best_params_.items(), key=operator.itemgetter(0))

    # print results
    out_text = '\t'.join([url.split('/')[-1][:-7],
                          'pop-cod',
                          str(i),
                          str(sorted_grid_params).replace('\n',','),
                          str(train_score_mse),
                          str(train_score_mae),
                          str(test_score_mse),
                          str(test_score_mae),
                          str(runtime)])

    data = [str(i), str(sorted_grid_params).replace('\n',','), str(train_score_mse), str(train_score_mae), str(test_score_mse), str(test_score_mae), str(runtime)]
    with open(url.split('/')[-1][:-16]+ ".csv", 'a+', newline='') as file:
        wr = csv.writer(file)
        wr.writerow(data)

    print(out_text)


HTTPError: ignored